# Queing Theory

## Usefull functions

littles_law (generic function with 1 method)

## M/M/s

In [13]:
function Cn(n,lambda,mu,s)
    if n <= s
        Cn = ((lambda/mu)^n)/n!
    else
        ((lambda/mu)^n)/(s!*s^(n-s))
    end
end

function P0(lambda,mu,s)
    a = 0
    for n in 0:s-1 
        a += 1/(((lambda/mu)^n)/factorial(n) + ((lambda/mu)^s)/factorial(s)*1/(1-lambda/(s*mu)))
    end
end

function Pn(n,lambda,mu,s)
    if n <= s
        (lambda/mu)^n/factorial(n)*P0(lambda,mu,s)
    else
        (lambda/mu)^n/(factorial(s)*s^(n-s))*P0(lambda,mu,s)
    end
end

function MMs(n,lambda,mu,s)
    pn = 0
    cn = 0
    p0 = 0
    if lambda < s*mu
        pn = Pn(n,lambda,mu,s)
        p0 = P0(lambda,mu,s)
    else
        cn = Cn(n,lambda,mu,s)
    end
    return cn,pn,p0
end


MMs (generic function with 1 method)

In [15]:
MMs(1,4,4,2)

MethodError: MethodError: no method matching *(::Float64, ::Nothing)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at C:\Users\rasmu\AppData\Local\Programs\Julia-1.7.2\share\julia\base\operators.jl:655
  *(::Union{Float16, Float32, Float64}, !Matched::BigFloat) at C:\Users\rasmu\AppData\Local\Programs\Julia-1.7.2\share\julia\base\mpfr.jl:406
  *(::Union{Real, Complex}, !Matched::Union{LinearAlgebra.Adjoint{var"#s859", var"#s8591"}, LinearAlgebra.Transpose{var"#s859", var"#s8591"}} where {var"#s859"<:Union{Real, Complex}, var"#s8591"<:(AbstractVector)}, !Matched::AbstractMatrix{<:Union{Real, Complex}}, !Matched::AbstractMatrix{<:Union{Real, Complex}}) at C:\Users\rasmu\AppData\Local\Programs\Julia-1.7.2\share\julia\stdlib\v1.7\LinearAlgebra\src\matmul.jl:1172
  ...

## Terms

### Balking

lambda_n: customer refusing to enter the system

### Renege

mu_n: customer enters the system, but leaves without being served